# Intro 

Previous Bayesian Theory of Mind models gave insights into mentalizing at the computational level, but not neccesarily the representations or algorithms underlying mental state inference. Previous models generally do not scale to large hypothesis spaces either.

What sorts of representations do people have of others' mental states? How do people simplify complex inference problems?

Recent work from Tamir & Thornton (2018; 2021) has mapped out certain abstract dimensions that organize people's neural representations of others' mental states (specifically emotions and other broad mental states, like "drunkeness" for instance). These dimensions of Valence, Rationality, and Social Impact comprise the 3D Mind Model. They have also mapped out abstract dimensions for observing others' actions (i.e. the ACT-FASTaxonomy).

Below I examine two Bayesian models which predict mental states given some action, or P(mental state|action). A higher dimensional model considers a vast list of mental states and actions, and the discrete probabilities which relate them. Alternatively, using Tamir & Thornton's abstract dimensions can offer a potentially more efficient way of searching this hypothesis space, by performaing inference over a low dimensional vector space, made up of the abstract dimensions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as sp
import pymc as pm

In [ ]:
dfPosteriors = pd.read_csv('posteriorRatings.csv')
dfLikelihoods = pd.read_csv('likelihoodRatings.csv')
dfPriors = pd.read_csv('priorRatings.csv')
actionCoordinates = pd.read_csv('pilotActions.csv')
stateCoordinates = pd.read_csv('pilotStates.csv')

stateCoordinates=stateCoordinates[['Valence', 'Social_Impact', 'Rationality-reverse-coded']].rename(index=stateCoordinates['States'])
actionCoordinates=actionCoordinates[['Abstraction', 'Animacy', 'Spiritualism', 'Food', 'Creation', 'Tradition']].rename(index=actionCoordinates['Action'])

# **High-Dimensional / Discrete Model**

In [ ]:
def priorTable(df, stim):
    priorDict = {}
    for state in list(stim):
        priorDict[state] = (sum([df['rating'][i] for i in range(len(df)) if df['stim1'][i] == state])/10)
    return priorDict

def likelihoodTable(df, states, actions):
    dfPost = pd.DataFrame(columns = states, index= actions)
    for act in actions:
        for state in states:
            num = 0
            for row in range(len(df)):
                if df['stim2'][row] == act and df['stim1'][row] == state:
                    if math.isnan(dfPost[state][act]):
                        dfPost[state][act] = df['rating'][row]
                        num = num + 1
                    elif num == 9:
                        dfPost[state][act] = (df['rating'][row]+dfPost[state][act])/10
                    else:
                        dfPost[state][act] = df['rating'][row]+dfPost[state][act]
                        num = num + 1
    return dfPost

In [ ]:
hi_dim_Likelihood = likelihoodTable(dfLikelihoods, stateCoordinates.index, actionCoordinates.index)
hi_dim_Prior = priorTable(dfPriors, stateCoordinates.index)

# **Low-Dimensional / Vector-Space Model**

## Prior

The prior is a Multivariate Normal distribution over the 3 mental state dimensions, fit from the Prior rating data. I use weighted datapoints to fit the distribution, the weights being people's probability ratings.

In [ ]:
def MVprior(dfPr, sCoord):
    #get datapoints (abstract mental state values) and weights 
    weights = np.array(dfPr['rating'])
    datapoints = np.array([sCoord.loc[state] for state in dfPr['stim1']])
    
    means = np.average(datapoints, axis=0, weights=weights) #mean vector     
    cov = np.cov(datapoints.T, aweights=weights) #covariance matrix
    return means, cov

prior_means, prior_cov = MVprior(dfPriors, stateCoordinates)

In [ ]:
priorMVN = sp.multivariate_normal(prior_means, prior_cov)

## Likelihoods

The likelihood requires some mapping from points in action space and points in mental state space. This mapping is unknown, so I show both a linear (separate multiple regression equations) approach and a non-linear (neural network) approach, which end up not being very different.

Specifically, I use PyMC to do Bayesian regression and a Bayesian neural network so the output for new predictions is a probability distribution (representing people's likelihood distribution).

To find these statistical mappings I use the data from the likelihood ratings. X and Y are the abstract values for actions and mental states respectively. These are used to fit the parameters for both the regression and neural network, but importantly, I use the probabiliy ratings people as "weights" on the datapoints. Instead of scaling the error like some do for weighted data in frequentist stats, I scale the log probability (logp) of that data under the model. Thus, higher probability ratings shift the model's parameters more during training. 

### Get x and y datapoints

In [ ]:
def getLikelihoodx_y(sCoord, aCoord, dfLik):

    x=[]
    y=[]
    
    for r in range(len(dfLik)):
        state = dfLik['stim1'][r]
        act = dfLik['stim2'][r]
        x.append([aCoord['Abstraction'][act], aCoord['Animacy'][act],
                  aCoord['Spiritualism'][act], aCoord['Food'][act], 
                  aCoord['Creation'][act], aCoord['Tradition'][act]])
        y.append([sCoord['Valence'][state], sCoord['Social_Impact'][state], 
                   sCoord['Rationality-reverse-coded'][state]])
        
    x = np.array(x)
    y = np.array(y)
    weights = dfLik['rating']+1
    return np.array(x), np.array(y), np.array(weights)

x, y, w = getLikelihoodx_y(stateCoordinates, actionCoordinates, dfLikelihoods)

Now the same, but for the posteriors we want to predict.

In [ ]:
def getPosteriorx_y(sCoord, aCoord, dfPost):
    
    x=[]
    y=[]
    
    for r in range(len(dfPost)):
        act = dfPost['stim1'][r]
        state = dfPost['stim2'][r]
        x.append([aCoord['Abstraction'][act], aCoord['Animacy'][act],
                  aCoord['Spiritualism'][act], aCoord['Food'][act], 
                  aCoord['Creation'][act], aCoord['Tradition'][act]])
        y.append([sCoord['Valence'][state], sCoord['Social_Impact'][state], 
                   sCoord['Rationality-reverse-coded'][state]])
        
    x = np.array(x)
    y = np.array(y)
    ratings = dfPost['rating']
    return np.array(x), np.array(y), np.array(ratings)

xP, yP, posteriorRatings = getPosteriorx_y(stateCoordinates, actionCoordinates, dfPosteriors)

### Linear Model
Fit the parameters of the linear model.

In [ ]:
with pm.Model() as linearModel:
    
    #model's data
    weights = pm.Data("weights", np.array([w, w, w]).T, mutable=False)
    x_obs = pm.MutableData('x_observation', x)
    y_obs = pm.MutableData('y_observation', y)  
    
    # define priors over parameters
    betas = pm.Normal('slopes_matrix', 0., 10., shape=(6, 3))
    beta0 = pm.Normal('intercepts', 0., 10., shape=(1,3))

    # predictions
    y_pred = pm.Deterministic('y_prediction', pm.math.dot(x_obs, betas) + beta0)
    
    sigmas = pm.HalfNormal('sigmas', 25., shape=3)
    
    obs = pm.Normal('observation', y_pred, sigmas, observed=y_obs)
    
    #Weighted datapoints 
    #note: pm.Potential doesn't affect later sampling predictions from model, only for infering parameters
    obs_weighted = pm.Potential('observation_weighted', 
                                weights*pm.logp(pm.Normal.dist(y_pred, sigmas), y_obs))

    
    #use MCMC to sample
    trace = pm.sample(draws=12000, tune=12000)

Now to make predictions for the likelihood distributions on the Posterior experiment trials we want to predict.

In [ ]:
actions =[actionCoordinates.loc[act] for act in actionCoordinates.index]

In [ ]:
with linearModel:
    pm.set_data({'x_observation': xP})
    predictive = pm.sample_posterior_predictive(trace)
    linearModel_predictions = np.mean(predictive['posterior_predictive']['observation'], axis=0)

### Non-linear Model
Same steps as for the linear model.

In [ ]:
n_hidden=6

with pm.Model() as nn_model:

    #Weight Datapoints
    weights = pm.Data("weights", np.array([w, w, w]).T, mutable=False) 

    #X and Y
    x_obs = pm.MutableData('x_obs', x)
    y_obs = pm.MutableData('y_obs', y)

    # Input -> Layer 1
    weights_1 = pm.Normal('w_1', mu=0, sigma=20,
                          shape=(6, n_hidden))
    b_01 = pm.Normal('intercept_1', 0., 25., shape=(1,n_hidden))
    acts_1 = pm.Deterministic('activations_1',
                              pm.math.tanh(pm.math.dot(x_obs, weights_1))+b_01)

    # Layer 1 -> Layer 2
    weights_2 = pm.Normal('w_2', mu=0, sigma=20,
                          shape=(n_hidden, n_hidden))
    b_02 = pm.Normal('intercept_2', 0., 25., shape=(1,n_hidden))
    acts_2 = pm.Deterministic('activations_2',
                              pm.math.tanh(pm.math.dot(acts_1, weights_2))+b_02)
    
    # Layer 2 -> Layer 3
    weights_3 = pm.Normal('w_3', mu=0, sigma=20,
                            shape=(n_hidden, 3))
    b_03 = pm.Normal('intercept_3', 0., 25., shape=(1,3))
    acts_3 = pm.Deterministic('activations_3',
                              pm.math.tanh(pm.math.dot(acts_2, weights_3)+b_03) 

    # Layer 3 -> Output Layer
    weights_out = pm.Normal('w_out', mu=0, sigma=20,
                            shape=(n_hidden, 3))
    b_04 = pm.Normal('intercept_4', 0., 25., shape=(1,3))
    acts_out = pm.Deterministic('activations_out',
                                pm.math.dot(acts_3, weights_out)+b_04) 

    # Define likelihood
    sigmas = pm.HalfNormal('sigmas', 50., shape=3)
    out = pm.Normal('out', acts_out, sigmas, observed=y_obs)
                              
    #Weighted datapoints
    out_weighted = pm.Potential('out_w', weights*pm.logp(pm.Normal.dist(acts_out, sd_dist), y_obs))

    #use MCMC to sample
    trace_nn = pm.sample(draws=5000, tune=5000)

In [ ]:
with nn_model:
    pm.set_data({'x_obs': xP})
    predictive_nn = pm.sample_posterior_predictive(trace_nn)
    nn_model_predictions = predictive_nn['posterior_predictive']['out']

In [ ]:
linearModel_predictions = np.mean(linearModel_predictions, axis=0)
nn_model_predictions = np.mean(nn_model_predictions, axis=0)

### Hybrid Model

In [ ]:
def getHybridx_y(sCoord, aCoord, dfLike):
    y1 = []
    y2 = []
    y3 = []
    actDict={}
    for a in aCoord['Action']:
        actDict[a] = []
    for r in range(len(dfLike)):
        sta = dfLike['stim1'][r]
        act = dfLike['stim2'][r]
        y1 = sCoord['Valence'][sta]
        y2 = sCoord['Social_Impact'][sta]
        y3 = sCoord['Rationality-reverse-coded'][sta]
        w = dfLike['rating'][r]
        actDict[act].append([y1,y2,y3,w])    
    
    return actDict

def getMVNsactDict(actDict):
    for act in actDict:
        Ws = [w[3] for w in actDict[act]]
        datapoints = np.array(actDict[act])[:,:3]
        av = np.average(datapoints, axis=0, weights=Ws)     
        cov = np.cov(datapoints.T, aweights=Ws)
        likelihoodMVN = sp.multivariate_normal(av, cov)
        actDict[act] = likelihoodMVN

    return actDict

In [ ]:
actDict = getHybridx_y(sCoord, aCoord, dfL)
hybridLikelihooodDict = getMVNsactDict(actDict)

# **Compute Posterior Predictions**

Combines the priors and likelihoods for each variation of the vector space model, and returns predictions for each posterior trial.

In [ ]:
def combine_LikPri_discretely(MS, stateDims, L, P, justL=False):
    listProbs = []
    for m in range(len(stateDims)):
        stateVector = [stateDims['Valence'][m],
           stateDims['Rationality-reverse-coded'][m],
           stateDims['Social_Impact'][m]]
        if (P == None) & (justL):
            prob = L.pdf(stateVector)
        else:
            prob = L.pdf(stateVector)*P.pdf(stateVector)
        listProbs.append(prob)
        if stateDims['States'][m] == MS:
            index = m                    
    normalizedProbs = [p/sum(listProbs) for p in listProbs]
    return normalizedProbs[index]

def modelCorrelation(human_ratings, model_probs):
    return sp.pearsonr(human_ratings, model_probs)

def runModel(posterior_df, stateDims, Lpredictions, priorMVN=None, modelType='nn', justLik=False):
    finalPreds = []
    if modelType == 'nn':    
        for k in range(len(posterior_df)):
            av = np.mean(Lpredictions[:,k,:], axis=0)     # (5000, 5760, 3)
            cov = np.cov(Lpredictions[:,k,:].T)
            likelihoodMVN = sp.multivariate_normal(av, cov)
            normedProb = combine_LikPri_discretely(posterior_df['stim2'][k], stateDims, likelihoodMVN, priorMVN, justL=justLik)
            finalPreds.append(normedProb)
    elif modelType == 'hybrid':
        for k in range(len(posterior_df)):
            likelihoodMVN = Lpredictions[posterior_df['stim1'][k]] #predictions is dictionary of MVNs
            normedProb = combine_LikPri_discretely(posterior_df['stim2'][k], stateDims, likelihoodMVN, priorMVN, justL=justLik)
            finalPreds.append(normedProb)

    results = modelCorrelation(posterior_df['rating'], finalPreds)
    print(results)
        
    return results, finalPreds

Now the same, but for the Discrete Model.

In [ ]:
def runDiscreteModel(df, lik, pri):
    predictions = []
    for row in range(len(df)):
        act = df['stim1'][row]
        sta = df['stim2'][row]
        pred = (lik[sta][act] * pri[sta])
        prediction = pred/(sum(
            [(lik[state][act] * pri[state]) for state in lik.columns]
        ))
        predictions.append(prediction)
    return sp.pearsonr(predictions, df['rating']), predictions

In [ ]:
aCoord = pd.read_csv('pilotActions.csv')
sCoord = pd.read_csv('pilotStates.csv')

sCoord=sCoord.rename(index=sCoord['States'])
aCoord=aCoord.rename(index=aCoord['Action'])

These functions below return 2 variables: an overall correlation coefficient and a list of predictions from that model for each trial. 

In [ ]:
#Linear
result_lm, pred_lm = runModel(dfPosteriors, sCoord, linearModel_predictions, 
                                   priorMVN = priorMVN, 
                                   modelType='nn', justLik=False) 

In [ ]:
#Non-linear
result_nn, pred_nn = runModel(dfPosteriors, sCoord, nn_model_predictions, 
                                   priorMVN = priorMVN, 
                                   modelType='nn', justLik=False) 

In [ ]:
#Hybrid
result_hm, pred_hm = runModel(dfPosteriors, sCoord, hybridLikelihooodDict,
                                   priorMVN = priorMVN,
                                   modelType='hybrid', justLik=False) 

In [ ]:
#Discrete
result_discrete, pred_discrete = runDiscreteModel(dfPosteriors, 
                                                  hi_dim_Likelihood, hi_dim_Prior) 